In [2]:
import numpy as np
from time import sleep
from matplotlib import pyplot as plt
from matplotlib import colors

from scaling_plot import read_dataframe_from_excel

%matplotlib inline

In [ ]:
class Results(object):
    def __init__(self, baseline_time, baseline_label, color):
        self.speedups = [1]
        self.labels = [baseline_label]
        self.colors = [color]
        self.baseline_time = baseline_time
        self.times = []
        
         # ugly code to handle Jupyter version differences
        try:
            # newer versions of TimeItResult contain the average time already
            self.times.append(self.baseline_time.average)
        except:
            # Failing that, older versions have a list of all times
            self.times.append(np.mean(self.baseline_time.all_runs))
        
    def speedup_plot(self, log_scale=False):
        ind = np.arange(len(self.speedups))
        fig, ax = plt.subplots(figsize=(12,8))
        
        for name, index, speedup, color in zip(self.labels, ind, self.speedups, self.colors):
            ax.bar(
                index,
                speedup, 
                width=0.5, 
                color=color,
                label=name,
                log=log_scale)
            
        ax.set_xticks(ind)
        ax.set_xticklabels(self.labels)
        ax.set_title('Speedup relative to {}'.format(self.labels[0]))
        ax.legend(self.labels)
        plt.show()
        return self
        
    def walltime_plot(self, log_scale=False):
        ind = np.arange(len(self.speedups))
        fig, ax = plt.subplots(figsize=(12,8))
        
        for name, index, walltime, color in zip(self.labels, ind, self.times, self.colors):
            ax.bar(
                index,
                walltime, 
                width=0.5, 
                color=color,
                label=name,
                log=log_scale)
            
        ax.set_xticks(ind)
        ax.set_xticklabels(self.labels)
        ax.set_title('Walltime')
        ax.legend(self.labels)
        plt.show()
        return self
        
    def add_time(self, time, label, color):
        
        baseline_avg = self.times[0]
        # ugly code to handle Jupyter version differences
        try:
            # newer versions of TimeItResult contain the average time already
            self.speedups.append(baseline_avg / time.average)
            self.times.append(time.average)
        except:
            # Failing that, older versions have a list of all times
            avg = np.mean(time.all_runs)
            self.speedups.append(baseline_avg / avg)
            self.times.append(avg)
            
        self.labels.append(label)
        self.colors.append(color)
        
        # return a reference to self to allow chaining
        return self

In [ ]:
times_baseline = %timeit -n 1 -r 1 -o results = sleep(0.5)

In [ ]:
results = Results(times_baseline, 'baseline', 'blue')

In [ ]:
t = %timeit -n 1 -r 1 -o results = sleep(0.2)
results.add_time(t, '1', 'green');

In [ ]:
t = %timeit -n 1 -r 1 -o results = sleep(0.1)
results.add_time(t, '2', 'yellow');

In [ ]:
results.speedup_plot();

In [ ]:
results.walltime_plot();

# Spreadsheet Version

In [20]:
category_column = 'version'# args.category_column
baseline_category = 'baseline'

usecols = [
    category_column,
    'walltime']

#if args.filter_column:
#    usecols.append(args.filter_column)

In [34]:
results = read_dataframe_from_excel(
    './examples/results.xls', #args.results_file,
    'categorical', #worksheet=args.worksheet_name,
    usecols=usecols)

# filter out incomplete results
results = results[results[category_column].notnull() & (results.walltime > 0)]

# apply the optional filter column
#if args.filter_column:
#    results = results[results[args.filter_column] > 0]

In [35]:
results

,version,walltime
0,baseline,13.1
1,first change,10.9
2,baseline,13.0


In [36]:
# if there are multiple times for each category then calculate the mean
results = results.groupby([category_column]).mean().reset_index()

In [37]:
results

,version,walltime
0,baseline,13.05
1,first change,10.90


In [38]:
t1 = float(results.loc[results[category_column] == baseline_category]['walltime'])
results['speedup'] = t1 / results['walltime']

In [39]:
results

,version,walltime,speedup
0,baseline,13.05,1.000000
1,first change,10.90,1.197248


In [43]:
list(results[category_column])

['baseline', 'first change']